In [2]:
import requests
import helper_import_path 
import Request_Analysis_JSON_helper as JHelper


#### Getting request movement data

In [ ]:
limit = 10000
offset = 310000
requests_movement_df = JHelper.get_movement_cpgrams(limit, offset)

# writing to csv
requests_movement_df.to_csv("../Data/req_movement_off_"+str(offset)+"_lim_"+str(limit)+".csv")
#removing the limit as extension. This is creating new files each time we run with different limit. Let it override.  
#requests_movement_df.to_csv("../Data/req_movement_details.csv")


#### Getting original request data

In [34]:
import Request_Analysis_JSON_helper as JHelper
limit=10000
offset = 250000
original_request_data = JHelper.get_original_cpgrams(limit,offset)
import removing_dup_reg_nos as remove_dup
original_request_data = remove_dup.removing_dup_reg_nos(original_request_data)
original_request_data.to_csv("../Data/original_req_off_"+str(offset)+"_lim_"+str(limit)+".csv")

The api request returned with resoponse code 200
No. of rows captured 10000 with offset =  250000
  SourceName closing_date comments country_name         diarydate  \
0        PMO   01-08-2019       NA        India  08-07-2019 21:02   
1        PMO   01-08-2019       NA        India  08-07-2019 21:02   
2        PMO   31-07-2019       NA        India  08-07-2019 21:04   
3        PMO   23-07-2019       NA        India  08-07-2019 21:09   
4        PMO   05-08-2019       NA        India  08-07-2019 21:11   

             distname                                ministry_department  \
0           Bagalkot                             Government of Karnataka   
1           Bagalkot                   Ministry of Labour and Employment   
2           Davangere  Department of Financial Services (Banking Divi...   
3  Udham Singh Nagar   Department of Financial Services (Banking Divi...   
4            Bilaspur              Ministry of Railways ( Railway Board)   

  rating ratingdate       regi

In [ ]:
#print("Unique reg id in Movement file",requests_movement_df['registration_no'].unique())
print("Printing number of unique reg ID in the movement data",len(requests_movement_df['registration_no'].unique()))
#print("Unique reg id in Original file",original_request_data['registration_no'].unique())
print("Printing unique reg id in the original data",len(original_request_data['registration_no'].unique()))

#### Selecting the data for shortlisted action_names

In [1]:
# This section o code reads the movement files and find out action_name = #EAOL 
# and removes other rows, and sotres in a new file

import pandas as pd

read_limit = 10000
read_offset = 200000

#read the file
data_request_movement_df = pd.read_csv("../Data/req_movement_off_"+str(read_offset)+"_lim_"+str(read_limit)+".csv")
#data_request_movement_df.head(5)



In [43]:
def selecting_req_data_for_action(data_request_movement_df,read_offset,read_limit,
                                  single_action =True,
                                  single_action_name='EXAMINED AT OUR LEVEL',
                                  selected_action_list=['INTERIM REPLY TO COMPLAINANT','CASE DISPOSED OF','CASE REPORT SENT AND DISPOSED LOCALLY ','EXAMINED AT OUR LEVEL']):

    if single_action:
        ## Selecting EAOL Only
        #drop other cols which are not required
        new_EAOL_df = data_request_movement_df.loc[(data_request_movement_df['action_name'] == single_action_name)]
        #new_df.head(20)


        #writing to a file
        new_EAOL_df.to_csv("../Data/req_movement_EAOL_off_"+str(read_offset)+"_lim_"+str(read_limit)+".csv")
    else:
        ## Selecting Multiple actions
        new_EAOL_df = data_request_movement_df.loc[(data_request_movement_df['action_name'].apply(lambda x:x in selected_action_list))]
        new_EAOL_df.to_csv("../Data/req_movement_sel_action_off_"+str(read_offset)+"_lim_"+str(read_limit)+".csv")
    return new_EAOL_df

In [78]:
import glob
import re
read_limit = 10000
selected_action_req_data = pd.DataFrame()
for filepath in glob.glob('./../Data/req_movement_off_'+'[0-9]*'+'_lim_10000.csv'):
    #print(filepath)
    m = re.search('./../Data/req_movement_off_(.+?)_lim_10000.csv', filepath)
    if m:
        read_offset = m.group(1)
    else:
        print("Could not find the file correctly.",filepath)
        continue
    #print(offset)
    
    temp = pd.read_csv(filepath)
    #'CASE REPORT SENT AND DISPOSED LOCALLY ',
    new_EAOL_df = selecting_req_data_for_action(temp,read_offset,read_limit,
                                  single_action =False,
                                  single_action_name='EXAMINED AT OUR LEVEL',
                                  selected_action_list=['INTERIM REPLY TO COMPLAINANT','CASE DISPOSED OF','EXAMINED AT OUR LEVEL'])
    new_EAOL_df.reset_index(drop=True,inplace=True)
    #print("Rows of interest",new_EAOL_df.shape)
    if selected_action_req_data.shape[0] == 0:
        selected_action_req_data = new_EAOL_df
    else:
        selected_action_req_data = selected_action_req_data.append(new_EAOL_df,ignore_index=True)
    


Rows of interest (1255, 8)
Rows of interest (1386, 8)
Rows of interest (1226, 8)
Rows of interest (1526, 8)
Rows of interest (1422, 8)
Rows of interest (1420, 8)
Rows of interest (1243, 8)
Rows of interest (2307, 8)
Rows of interest (1242, 8)
Rows of interest (1648, 8)
Rows of interest (1288, 8)
Rows of interest (1223, 8)
Rows of interest (1336, 8)
Rows of interest (1865, 8)
Rows of interest (1306, 8)
Rows of interest (1220, 8)
Rows of interest (1212, 8)
Rows of interest (1421, 8)
Rows of interest (1725, 8)
Rows of interest (1464, 8)
Rows of interest (1250, 8)
Rows of interest (1262, 8)


In [79]:
selected_action_req_data.columns

Index(['Unnamed: 0', 'action_name', 'action_srno', 'date_of_action',
       'org_name', 'org_name2', 'registration_no', 'remarks'],
      dtype='object')

In [80]:
selected_action_req_data.shape

(31247, 8)

In [81]:
#This section of code reds the org_name from *EAOL*.csv s and finds out 'Apex Ministry/Dept/State'
# and 'Parent of Organisation' from 'NodalOfficer_Details.csv' file and appends it to the same file

#read the file 
filepath_NodalOfficer_Details = "./../Data/" 
filename_NodalOfficer_Details = "NodalOfficer_Details.csv"
print(filepath_NodalOfficer_Details + filename_NodalOfficer_Details)
data_NodalOfficer_Details = pd.read_csv(filepath_NodalOfficer_Details + filename_NodalOfficer_Details, header=0, encoding = 'unicode_escape' )

./../Data/NodalOfficer_Details.csv


In [82]:
#data_NodalOfficer_Details.columns

#### Converting the orgname data to corresponding Apex Ministry name and storing the same in the file.

In [83]:
#find out what is the 'Apex Ministry/Dept/State' and 'Parent of Organisation'
#print(new_EAOL_df['org_name'], " : ", data_NodalOfficer_Details['org_name'])
new_OrgName_df = []
new_Apex_org = []
no_department_found = []
for org_name in selected_action_req_data['org_name']:
    #print(org_name)
    temp = data_NodalOfficer_Details.loc[(data_NodalOfficer_Details['org_name']== org_name),['Apex Ministry/Dept/State','Parent of Organisation']]
    if len(temp) == 0:
        #need to store and continue : 
        #This means we could not find the corresponding apex dept name in the NodelOfficer file
        no_department_found.append(org_name)
        continue
    if len(temp['Apex Ministry/Dept/State'].unique()) >1:
     # to know which ones have multiple Parent dept name.
        print(org_name)
        #print(temp)
        print("More than one values")
        #break
    new_OrgName_df.append(temp['Parent of Organisation'].values)
    new_Apex_org.append(temp['Apex Ministry/Dept/State'].values[0])
    #new_OrgName_df.append(data_NodalOfficer_Details[(data_NodalOfficer_Details['org_name']== org_name)], ignore_index = True)

State Bank of India
More than one values
State Bank of India
More than one values
Bank of Baroda
More than one values
Bank of Baroda
More than one values
INCOME TAX
More than one values
State Bank of India
More than one values
State Bank of India
More than one values
State Bank of India
More than one values
Bank of Baroda
More than one values
State Bank of India
More than one values
State Bank of India
More than one values
Bank of Maharashtra
More than one values
Bank of Maharashtra
More than one values
Central Bank of India
More than one values
Central Bank of India
More than one values
Department of Revenue
More than one values
State Bank of India
More than one values
State Bank of India
More than one values
Government of Uttar Pradesh
More than one values
Government of Uttar Pradesh
More than one values
Punjab National Bank
More than one values
State Bank of India
More than one values
State Bank of India
More than one values
INCOME TAX
More than one values
State Bank of India
More t

State Bank of India
More than one values
Punjab National Bank
More than one values
State Bank of India
More than one values
State Bank of India
More than one values


In [84]:
temp['Apex Ministry/Dept/State'].unique()

array(['Central Board of Direct Taxes (Income Tax)'], dtype=object)

In [85]:
print(len(new_OrgName_df))
print(len(new_Apex_org))
selected_action_req_data.shape

31247
31247


(31247, 8)

In [86]:
selected_action_req_data['Parent_org_name'] = new_OrgName_df
selected_action_req_data['Apex_org_name'] = new_Apex_org

In [90]:
## This file stores the 
## Caution : Please ensure you want to overwrite this file.
selected_action_req_data.to_csv("./../Data/processed_selected_action.csv", header=True,index=False)

In [91]:
selected_action_req_data.columns

Index(['Unnamed: 0', 'action_name', 'action_srno', 'date_of_action',
       'org_name', 'org_name2', 'registration_no', 'remarks',
       'Parent_org_name', 'Apex_org_name'],
      dtype='object')

In [92]:
len(selected_action_req_data['registration_no'].unique())

17666

In [93]:
selected_action_req_data.shape

(31247, 10)

In [96]:
reg_group = selected_action_req_data.groupby('registration_no')

In [100]:
reg_group.get_group('')